In [4]:
import random

class Process:
    def __init__(self, pid, arrival_time, burst_time):
        self.pid = pid  # Process ID
        self.arrival_time = arrival_time  # Arrival time of the process
        self.burst_time = burst_time  # CPU burst time
        self.remaining_time = burst_time  # Remaining time to finish the process
        self.wait_time = 0  # Total wait time
        self.turnaround_time = 0  # Turnaround time

def round_robin(processes, quantum):
    time = 0
    queue = []
    completed_processes = []
    processes.sort(key=lambda p: p.arrival_time)  # Sort processes by arrival time
    i = 0  # Index to track new arrivals

    while queue or i < len(processes):
        # Add processes that have arrived to the queue
        while i < len(processes) and processes[i].arrival_time <= time:
            queue.append(processes[i])
            i += 1

        if queue:
            process = queue.pop(0)
            
            if process.remaining_time > quantum:
                time += quantum
                process.remaining_time -= quantum
                # Add processes that arrive during this quantum
                while i < len(processes) and processes[i].arrival_time <= time:
                    queue.append(processes[i])
                    i += 1
                queue.append(process)  # Put the process back in the queue if it's not finished
            else:
                time += process.remaining_time
                process.wait_time = time - process.burst_time - process.arrival_time
                process.turnaround_time = time - process.arrival_time
                process.remaining_time = 0
                completed_processes.append(process)
        else:
            # If no process is in the queue, jump to the next process arrival time
            time = processes[i].arrival_time

    return completed_processes

original_process_list = [
    Process(pid='P1', arrival_time=21, burst_time=10),
    Process(pid='P2', arrival_time=12, burst_time=15),
    Process(pid='P3', arrival_time=8, burst_time=25),
    Process(pid='P4', arrival_time=10, burst_time=55),
    Process(pid='P5', arrival_time=12, burst_time=65),
    Process(pid='P6', arrival_time=15, burst_time=60),
]

# Manual input for quantum time
quantum = int(input("Enter the quantum time: "))

# Run Round Robin algorithm
completed = round_robin(original_process_list, quantum)

# Calculate average wait time and turnaround time
total_wait_time = sum(p.wait_time for p in completed)
total_turnaround_time = sum(p.turnaround_time for p in completed)
average_wait_time = total_wait_time / len(completed)
average_turnaround_time = total_turnaround_time / len(completed)

# Printing the results
print("\nPID\tArrival Time\tBurst Time\tWait Time\tTurnaround Time")
for process in completed:
    print(f"{process.pid}\t{process.arrival_time}\t\t{process.burst_time}\t\t{process.wait_time}\t\t{process.turnaround_time}")

# Print average wait time and average turnaround time
print(f"\nAverage Wait Time: {average_wait_time:.2f}")
print(f"Average Turnaround Time: {average_turnaround_time:.2f}")

Enter the quantum time:  5



PID	Arrival Time	Burst Time	Wait Time	Turnaround Time
P1	21		10		47		57
P2	12		15		56		71
P3	8		25		80		105
P4	10		55		148		203
P6	15		60		158		218
P5	12		65		161		226

Average Wait Time: 108.33
Average Turnaround Time: 146.67


In [5]:
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd
import statistics

class Process:
    def __init__(self, pid, arrival_time, burst_time):
        self.pid = pid
        self.arrival_time = arrival_time
        self.burst_time = burst_time
        self.remaining_time = burst_time
        self.waiting_time = 0
        self.turnaround_time = 0
        self.completed = False
        self.cluster_label = None  # Process'in bağlı olduğu küme

# Define the original process list
original_process_list = [
    Process(pid='P1', arrival_time=21, burst_time=10),
    Process(pid='P2', arrival_time=12, burst_time=15),
    Process(pid='P3', arrival_time=8, burst_time=25),
    Process(pid='P4', arrival_time=10, burst_time=55),
    Process(pid='P5', arrival_time=12, burst_time=65),
    Process(pid='P6', arrival_time=15, burst_time=60)
]

# KMeans ile kümeleme (k=2)
burst_times = np.array([[p.burst_time] for p in original_process_list])
kmeans = KMeans(n_clusters=2, random_state=0).fit(burst_times)
labels = kmeans.labels_

# Her bir process'in küme etiketini ve kümeye göre time quantum'unu belirleyelim
cluster_quantum = {}
for idx, process in enumerate(original_process_list):
    process.cluster_label = labels[idx]
    # Kümedeki en büyük burst time'ı küme için time quantum olarak belirle
    cluster_burst_times = [p.burst_time for p in original_process_list if p.cluster_label == process.cluster_label]
    #cluster_quantum[process.cluster_label] = int(statistics.mean(cluster_burst_times))
    cluster_quantum[process.cluster_label] = max(cluster_burst_times)

# Round Robin algoritması
time_elapsed = 0
process_queue = []
gantt_chart = []

while any(p.remaining_time > 0 for p in original_process_list):
    for process in original_process_list:
        if process.remaining_time > 0 and process.arrival_time <= time_elapsed and process not in process_queue:
            process_queue.append(process)
    if not process_queue:
        time_elapsed += 1
        continue

    process = process_queue.pop(0)
    time_quantum = cluster_quantum[process.cluster_label]
    if process.remaining_time <= time_quantum:
        
        time_elapsed += process.remaining_time
        gantt_chart.append((process.pid, time_elapsed))
        process.turnaround_time = time_elapsed - process.arrival_time
        process.waiting_time = process.turnaround_time - process.burst_time
        process.remaining_time = 0
    else:
        process.remaining_time -= time_quantum
        time_elapsed += time_quantum
        gantt_chart.append((process.pid, time_elapsed))
        process_queue.append(process)

# Tabloyu oluştur ve çıktıyı göster
table_data = {
    "PID": [p.pid for p in original_process_list],
    "Arrival Time": [p.arrival_time for p in original_process_list],
    "Burst Time": [p.burst_time for p in original_process_list],
    "Cluster": [p.cluster_label for p in original_process_list],
    "Quantum Time": [cluster_quantum[p.cluster_label] for p in original_process_list],
    "Wait Time": [p.waiting_time for p in original_process_list],
    "Turnaround Time": [p.turnaround_time for p in original_process_list]
}

df = pd.DataFrame(table_data)
print(df)

# Ortalama bekleme ve dönüş sürelerini hesapla
avg_waiting_time = sum([p.waiting_time for p in original_process_list]) / len(original_process_list)
avg_turnaround_time = sum([p.turnaround_time for p in original_process_list]) / len(original_process_list)

print("Gantt Chart:", [(pid, time) for pid, time in gantt_chart])
print("\nAverage Waiting Time:", avg_waiting_time)
print("Average Turnaround Time:", avg_turnaround_time)

  PID  Arrival Time  Burst Time  Cluster  Quantum Time  Wait Time  \
0  P1            21          10        1            25         12   
1  P2            12          15        1            25         31   
2  P3             8          25        1            25          0   
3  P4            10          55        0            65         48   
4  P5            12          65        0            65        101   
5  P6            15          60        0            65        163   

   Turnaround Time  
0               22  
1               46  
2               25  
3              103  
4              166  
5              223  
Gantt Chart: [('P3', 33), ('P1', 43), ('P2', 58), ('P4', 113), ('P5', 178), ('P6', 238)]

Average Waiting Time: 59.166666666666664
Average Turnaround Time: 97.5


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


In [7]:
from sklearn.cluster import DBSCAN
import numpy as np
import pandas as pd

class Process:
    def __init__(self, pid, arrival_time, burst_time):
        self.pid = pid
        self.arrival_time = arrival_time
        self.burst_time = burst_time
        self.remaining_time = burst_time
        self.waiting_time = 0
        self.turnaround_time = 0
        self.completed = False
        self.cluster_label = None
        self.quantum_time = None  # Quantum time for each process

def dbscan_clustering(process_list, eps=10, min_samples=2):
    burst_times = np.array([[p.burst_time] for p in process_list])
    dbscan = DBSCAN(eps=eps, min_samples=min_samples).fit(burst_times)
    labels = dbscan.labels_

    # Assign cluster labels and quantum times
    for idx, process in enumerate(process_list):
        process.cluster_label = labels[idx]
        if process.cluster_label != -1:
            # Get max burst time in the cluster as quantum time
            cluster_burst_times = [p.burst_time for p in process_list if p.cluster_label == process.cluster_label]
            process.quantum_time = max(cluster_burst_times)
        else:
            # Outliers get a default quantum time
            process.quantum_time = 5
    
    return process_list

def round_robin_scheduling(process_list):
    time_elapsed = 0
    process_queue = []
    gantt_chart = []

    while any(p.remaining_time > 0 for p in process_list):
        for process in process_list:
            if process.remaining_time > 0 and process.arrival_time <= time_elapsed and process not in process_queue:
                process_queue.append(process)

        if not process_queue:
            time_elapsed += 1
            continue

        process = process_queue.pop(0)
        time_quantum = process.quantum_time
        
        if process.remaining_time <= time_quantum:
            time_elapsed += process.remaining_time
            gantt_chart.append((process.pid, time_elapsed))
            process.turnaround_time = time_elapsed - process.arrival_time
            process.waiting_time = process.turnaround_time - process.burst_time
            process.remaining_time = 0
            process.completed = True
        else:
            process.remaining_time -= time_quantum
            time_elapsed += time_quantum
            gantt_chart.append((process.pid, time_elapsed))
            process_queue.append(process)
    
    return gantt_chart

# Define the original process list
original_process_list = [
    Process(pid='P1', arrival_time=21, burst_time=10),
    Process(pid='P2', arrival_time=12, burst_time=15),
    Process(pid='P3', arrival_time=8, burst_time=25),
    Process(pid='P4', arrival_time=10, burst_time=55),
    Process(pid='P5', arrival_time=12, burst_time=65),
    Process(pid='P6', arrival_time=15, burst_time=60)
]

# Apply DBSCAN clustering to assign quantum times
original_process_list = dbscan_clustering(original_process_list)

# Perform Round Robin scheduling
gantt_chart = round_robin_scheduling(original_process_list)

# Display the results in a DataFrame
table_data = {
    "PID": [p.pid for p in original_process_list],
    "Arrival Time": [p.arrival_time for p in original_process_list],
    "Burst Time": [p.burst_time for p in original_process_list],
    "Cluster": [p.cluster_label for p in original_process_list],
    "Quantum Time": [p.quantum_time for p in original_process_list],
    "Wait Time": [p.waiting_time for p in original_process_list],
    "Turnaround Time": [p.turnaround_time for p in original_process_list],
    "Completed": [p.completed for p in original_process_list]
}

df = pd.DataFrame(table_data)
print(df)

# Calculate average waiting and turnaround times
avg_waiting_time = sum([p.waiting_time for p in original_process_list]) / len(original_process_list)
avg_turnaround_time = sum([p.turnaround_time for p in original_process_list]) / len(original_process_list)

print("Gantt Chart:", gantt_chart)
print("\nAverage Waiting Time:", avg_waiting_time)
print("Average Turnaround Time:", avg_turnaround_time)


  PID  Arrival Time  Burst Time  Cluster  Quantum Time  Wait Time  \
0  P1            21          10        0            10         12   
1  P2            12          15        0            15         31   
2  P3             8          25        0            25          0   
3  P4            10          55        1            55         48   
4  P5            12          65        1            65        101   
5  P6            15          60        1            65        163   

   Turnaround Time  Completed  
0               22       True  
1               46       True  
2               25       True  
3              103       True  
4              166       True  
5              223       True  
Gantt Chart: [('P3', 33), ('P1', 43), ('P2', 58), ('P4', 113), ('P5', 178), ('P6', 238)]

Average Waiting Time: 59.166666666666664
Average Turnaround Time: 97.5


In [3]:
import random

class Process:
    def __init__(self, pid, arrival_time, burst_time):
        self.pid = pid  # Process ID
        self.arrival_time = arrival_time  # Arrival time of the process
        self.burst_time = burst_time  # CPU burst time
        self.remaining_time = burst_time  # Remaining time to finish the process
        self.wait_time = 0  # Total wait time
        self.turnaround_time = 0  # Turnaround time

def round_robin(processes, quantum):
    time = 0
    queue = []
    completed_processes = []
    processes.sort(key=lambda p: p.arrival_time)  # Sort processes by arrival time
    i = 0  # Index to track new arrivals

    while queue or i < len(processes):
        # Add processes that have arrived to the queue
        while i < len(processes) and processes[i].arrival_time <= time:
            queue.append(processes[i])
            i += 1

        if queue:
            process = queue.pop(0)
            
            if process.remaining_time > quantum:
                time += quantum
                process.remaining_time -= quantum
                # Add processes that arrive during this quantum
                while i < len(processes) and processes[i].arrival_time <= time:
                    queue.append(processes[i])
                    i += 1
                queue.append(process)  # Put the process back in the queue if it's not finished
            else:
                time += process.remaining_time
                process.wait_time = time - process.burst_time - process.arrival_time
                process.turnaround_time = time - process.arrival_time
                process.remaining_time = 0
                completed_processes.append(process)
        else:
            # If no process is in the queue, jump to the next process arrival time
            time = processes[i].arrival_time

    return completed_processes

original_process_list = [
    Process(pid='P1', arrival_time=21, burst_time=10),
    Process(pid='P2', arrival_time=12, burst_time=15),
    Process(pid='P3', arrival_time=8, burst_time=25),
    Process(pid='P4', arrival_time=10, burst_time=55),
    Process(pid='P5', arrival_time=12, burst_time=65),
    Process(pid='P6', arrival_time=15, burst_time=60),
    Process(pid='P7', arrival_time=5, burst_time=120)
]

# Manual input for quantum time
quantum = int(input("Enter the quantum time: "))

# Run Round Robin algorithm
completed = round_robin(original_process_list, quantum)

# Calculate average wait time and turnaround time
total_wait_time = sum(p.wait_time for p in completed)
total_turnaround_time = sum(p.turnaround_time for p in completed)
average_wait_time = total_wait_time / len(completed)
average_turnaround_time = total_turnaround_time / len(completed)

# Printing the results
print("\nPID\tArrival Time\tBurst Time\tWait Time\tTurnaround Time")
for process in completed:
    print(f"{process.pid}\t{process.arrival_time}\t\t{process.burst_time}\t\t{process.wait_time}\t\t{process.turnaround_time}")

# Print average wait time and average turnaround time
print(f"\nAverage Wait Time: {average_wait_time:.2f}")
print(f"Average Turnaround Time: {average_turnaround_time:.2f}")

Enter the quantum time:  5



PID	Arrival Time	Burst Time	Wait Time	Turnaround Time
P1	21		10		59		69
P2	12		15		73		88
P3	8		25		107		132
P4	10		55		200		255
P6	15		60		220		280
P5	12		65		228		293
P7	5		120		230		350

Average Wait Time: 159.57
Average Turnaround Time: 209.57


In [2]:
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd
import statistics

class Process:
    def __init__(self, pid, arrival_time, burst_time):
        self.pid = pid
        self.arrival_time = arrival_time
        self.burst_time = burst_time
        self.remaining_time = burst_time
        self.waiting_time = 0
        self.turnaround_time = 0
        self.completed = False
        self.cluster_label = None  # Process'in bağlı olduğu küme

# Define the original process list
original_process_list = [
    Process(pid='P1', arrival_time=21, burst_time=10),
    Process(pid='P2', arrival_time=12, burst_time=15),
    Process(pid='P3', arrival_time=8, burst_time=25),
    Process(pid='P4', arrival_time=10, burst_time=55),
    Process(pid='P5', arrival_time=12, burst_time=65),
    Process(pid='P6', arrival_time=15, burst_time=60),
    Process(pid='P7', arrival_time=5, burst_time=120)
]

# KMeans ile kümeleme (k=2)
burst_times = np.array([[p.burst_time] for p in original_process_list])
kmeans = KMeans(n_clusters=2, random_state=0).fit(burst_times)
labels = kmeans.labels_

# Her bir process'in küme etiketini ve kümeye göre time quantum'unu belirleyelim
cluster_quantum = {}
for idx, process in enumerate(original_process_list):
    process.cluster_label = labels[idx]
    # Kümedeki en büyük burst time'ı küme için time quantum olarak belirle
    cluster_burst_times = [p.burst_time for p in original_process_list if p.cluster_label == process.cluster_label]
    #cluster_quantum[process.cluster_label] = int(statistics.mean(cluster_burst_times))
    cluster_quantum[process.cluster_label] = max(cluster_burst_times)

# Round Robin algoritması
time_elapsed = 0
process_queue = []
gantt_chart = []

while any(p.remaining_time > 0 for p in original_process_list):
    for process in original_process_list:
        if process.remaining_time > 0 and process.arrival_time <= time_elapsed and process not in process_queue:
            process_queue.append(process)
    if not process_queue:
        time_elapsed += 1
        continue

    process = process_queue.pop(0)
    time_quantum = cluster_quantum[process.cluster_label]
    if process.remaining_time <= time_quantum:
        
        time_elapsed += process.remaining_time
        gantt_chart.append((process.pid, time_elapsed))
        process.turnaround_time = time_elapsed - process.arrival_time
        process.waiting_time = process.turnaround_time - process.burst_time
        process.remaining_time = 0
    else:
        process.remaining_time -= time_quantum
        time_elapsed += time_quantum
        gantt_chart.append((process.pid, time_elapsed))
        process_queue.append(process)

# Tabloyu oluştur ve çıktıyı göster
table_data = {
    "PID": [p.pid for p in original_process_list],
    "Arrival Time": [p.arrival_time for p in original_process_list],
    "Burst Time": [p.burst_time for p in original_process_list],
    "Cluster": [p.cluster_label for p in original_process_list],
    "Quantum Time": [cluster_quantum[p.cluster_label] for p in original_process_list],
    "Wait Time": [p.waiting_time for p in original_process_list],
    "Turnaround Time": [p.turnaround_time for p in original_process_list]
}

df = pd.DataFrame(table_data)
print(df)

# Ortalama bekleme ve dönüş sürelerini hesapla
avg_waiting_time = sum([p.waiting_time for p in original_process_list]) / len(original_process_list)
avg_turnaround_time = sum([p.turnaround_time for p in original_process_list]) / len(original_process_list)

print("Gantt Chart:", [(pid, time) for pid, time in gantt_chart])
print("\nAverage Waiting Time:", avg_waiting_time)
print("Average Turnaround Time:", avg_turnaround_time)

  PID  Arrival Time  Burst Time  Cluster  Quantum Time  Wait Time  \
0  P1            21          10        0            65        104   
1  P2            12          15        0            65        123   
2  P3             8          25        0            65        142   
3  P4            10          55        0            65        165   
4  P5            12          65        0            65        218   
5  P6            15          60        0            65        280   
6  P7             5         120        1           120          0   

   Turnaround Time  
0              114  
1              138  
2              167  
3              220  
4              283  
5              340  
6              120  
Gantt Chart: [('P7', 125), ('P1', 135), ('P2', 150), ('P3', 175), ('P4', 230), ('P5', 295), ('P6', 355)]

Average Waiting Time: 147.42857142857142
Average Turnaround Time: 197.42857142857142


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


In [1]:
from sklearn.cluster import DBSCAN
import numpy as np
import pandas as pd

class Process:
    def __init__(self, pid, arrival_time, burst_time):
        self.pid = pid
        self.arrival_time = arrival_time
        self.burst_time = burst_time
        self.remaining_time = burst_time
        self.waiting_time = 0
        self.turnaround_time = 0
        self.completed = False
        self.cluster_label = None
        self.quantum_time = None  # Quantum time for each process

def dbscan_clustering(process_list, eps=10, min_samples=2):
    burst_times = np.array([[p.burst_time] for p in process_list])
    dbscan = DBSCAN(eps=eps, min_samples=min_samples).fit(burst_times)
    labels = dbscan.labels_

    # Assign cluster labels and quantum times
    for idx, process in enumerate(process_list):
        process.cluster_label = labels[idx]
        if process.cluster_label != -1:
            # Get max burst time in the cluster as quantum time
            cluster_burst_times = [p.burst_time for p in process_list if p.cluster_label == process.cluster_label]
            process.quantum_time = max(cluster_burst_times)
        else:
            # Outliers get a default quantum time
            process.quantum_time = 5
    
    return process_list

def round_robin_scheduling(process_list):
    time_elapsed = 0
    process_queue = []
    gantt_chart = []

    while any(p.remaining_time > 0 for p in process_list):
        for process in process_list:
            if process.remaining_time > 0 and process.arrival_time <= time_elapsed and process not in process_queue:
                process_queue.append(process)

        if not process_queue:
            time_elapsed += 1
            continue

        process = process_queue.pop(0)
        time_quantum = process.quantum_time
        
        if process.remaining_time <= time_quantum:
            time_elapsed += process.remaining_time
            gantt_chart.append((process.pid, time_elapsed))
            process.turnaround_time = time_elapsed - process.arrival_time
            process.waiting_time = process.turnaround_time - process.burst_time
            process.remaining_time = 0
            process.completed = True
        else:
            process.remaining_time -= time_quantum
            time_elapsed += time_quantum
            gantt_chart.append((process.pid, time_elapsed))
            process_queue.append(process)
    
    return gantt_chart

# Define the original process list
original_process_list = [
    Process(pid='P1', arrival_time=21, burst_time=10),
    Process(pid='P2', arrival_time=12, burst_time=15),
    Process(pid='P3', arrival_time=8, burst_time=25),
    Process(pid='P4', arrival_time=10, burst_time=55),
    Process(pid='P5', arrival_time=12, burst_time=65),
    Process(pid='P6', arrival_time=15, burst_time=60),
    Process(pid='P7', arrival_time=5, burst_time=120)
]

# Apply DBSCAN clustering to assign quantum times
original_process_list = dbscan_clustering(original_process_list)

# Perform Round Robin scheduling
gantt_chart = round_robin_scheduling(original_process_list)

# Display the results in a DataFrame
table_data = {
    "PID": [p.pid for p in original_process_list],
    "Arrival Time": [p.arrival_time for p in original_process_list],
    "Burst Time": [p.burst_time for p in original_process_list],
    "Cluster": [p.cluster_label for p in original_process_list],
    "Quantum Time": [p.quantum_time for p in original_process_list],
    "Wait Time": [p.waiting_time for p in original_process_list],
    "Turnaround Time": [p.turnaround_time for p in original_process_list],
    "Completed": [p.completed for p in original_process_list]
}

df = pd.DataFrame(table_data)
print(df)

# Calculate average waiting and turnaround times
avg_waiting_time = sum([p.waiting_time for p in original_process_list]) / len(original_process_list)
avg_turnaround_time = sum([p.turnaround_time for p in original_process_list]) / len(original_process_list)

print("Gantt Chart:", gantt_chart)
print("\nAverage Waiting Time:", avg_waiting_time)
print("Average Turnaround Time:", avg_turnaround_time)


  PID  Arrival Time  Burst Time  Cluster  Quantum Time  Wait Time  \
0  P1            21          10        0            10        219   
1  P2            12          15        0            15         88   
2  P3             8          25        0            25          7   
3  P4            10          55        1            55         30   
4  P5            12          65        1            65        103   
5  P6            15          60        1            65        165   
6  P7             5         120       -1             5        230   

   Turnaround Time  Completed  
0              229       True  
1              103       True  
2               32       True  
3               85       True  
4              168       True  
5              225       True  
6              350       True  
Gantt Chart: [('P7', 10), ('P7', 15), ('P3', 40), ('P4', 95), ('P7', 100), ('P2', 115), ('P5', 180), ('P6', 240), ('P1', 250), ('P7', 255), ('P7', 260), ('P7', 265), ('P7', 270), ('P7', 275),